# Setup

In [ ]:
!pip install openai

     |████████████████████████████████| 42 kB 282 kB/s 
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
import os
import openai

ModuleNotFoundError: ignored

In [ ]:
# Enter API Key Here
openai.api_key = ""

# Pretexts for Fine-tuning

In [ ]:
pretext_tweets = '''
Tweet: The present strategy of large scale vaccination in rich countries and a shamefaced herdimmunity approach is a desperate gamble.
Hashtag: #COVID19

Tweet: Some Humans are so sick in the head I refuse to believe I am part of the same species.
Hashtag: #COVID19

Tweet: How Brian Pallister, Heather Stefanson and Obby Khan all used a variation of this tagline when they were asked about the PC government's pandemic response.
Hashtag: #COVID19"

Tweet: Some truths about covid19. It may not be what you want to hear, but it is what you NEED to hear.
Hashtag: #COVID19

Tweet: The fact that Brittany Griner even has to go to Russia in the first place to get paid a decent salary is the root problem. Imagine if this was Lebron James. This would be the biggest story in sports history. Respect and pay our women here at home
Hashtag: #NBA

Tweet: Michael Jordan | Bulls  vs Hawks | 1995 | 32pts, HE's BACK 
Hashtag: #NBA

Tweet: NBA "This Is My Vengeance!" MOMENTS includes players such as LeBron James, Kyrie Irving, Stephen Curry, Kevin Durant, Ja... 
Hashtag: #NBA
'''

In [ ]:
pretext_tweet_insentence = '''
Tweet: Tesla stocks are about to skyrocket
Hashtagged Tweet: #Tesla stocks are about to skyrocket #ElonMusk

Tweet: Stephen Curry is the greatest basketball player
Hashtagged Tweet: #StephenCurry is the greatest basketball player #NBA #basketball

Tweet: Is History repeating itself?!
Hashtagged Tweet: Is History repeating itself?! #dontNormalizeHate
'''

# Image to Text

In [ ]:
!pip install transformers command
!pip install easyocr
!pip install opencv-python-headless==4.5.2.52

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
import torchvision
import requests
from PIL import Image
import easyocr

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
reader = easyocr.Reader(['en'])

In [ ]:
def predict_step(images):
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [ ]:
def image2text(url):
  image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
  ocr = reader.readtext(url)
  ocr_text = ""
  for i in range(len(ocr)):
    ocr_text += " " + ocr[i][-2]
  des = predict_step([image])[0]
  return {'description': des, 'OCR': ocr_text}

# Inputs

In [ ]:
image_input = '''
Tweet Text: 
Image Description: an advertisement for a television show
Image OCR: On THIS DAH 14 YEARS AGO WARRIORS NATION STEPH CURRV INTPODUCED HIHSELF TO THE IORLD HND SCORED 40 POIITS AGHIIST HOHZHca CLUTCHPOINTS
Hashtags:'''

In [ ]:
def gen_hashtags(tweet):
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=pretext_tweets+"\n"+tweet+"\nHashtags:",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["\n"]
  )
  return response["choices"][0]["text"]

In [ ]:
gen_hashtags("\nTweet: I love Drake and DJ Khalid\nHashtags:")

' #Drake, #DJKhalid'

In [ ]:
def gen_insentence_hashtags(tweet):
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=pretext_tweet_insentence+"\n"+tweet+"\nHashttagged Tweet:",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["\n"]
  )
  return response["choices"][0]["text"]

In [ ]:
gen_insentence_hashtags("\nTweet: I love Drake and DJ Khalid\nHashtags:")

' I love #Drake and #DJKhalid'

In [ ]:
gen_insentence_hashtags("\nTweet: Putin needs to stop his attack on Ukraine\nHashtags:")

' #Putin needs to stop his attack on #Ukraine #Crimea'

In [ ]:
gen_insentence_hashtags("Today has been a teleporting type day. 3 cities, 1 day.")

' Today has been a teleporting type day. 3 cities, 1 day. #travel'

In [ ]:
def gen_image_hashtags(tweet, image_text):
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="Tweet: " + tweet + "\nImage Description: " + image_text['description'] + "\nImage OCR: " + image_text['OCR'] + "\nHashtags:",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["\n"]
  )
  return response["choices"][0]["text"]

In [ ]:
image_text = image2text("https://pbs.twimg.com/media/FOYTNj3UcAEjzq9?format=jpg&name=medium")

In [ ]:
gen_image_hashtags("", image_text), image_text

(' #Warriors #StephCurry',
 {'OCR': ' On THIS DAH 14 YEARS AGO WARRIORS NATION STEPH CURRV ITRODUCED HIISELF TO THE IORLD HND SCORED 40 POIITS AGHIIST HOHZHca CLUTCHPOINTS VAldon:',
  'description': 'an advertisement for a television show'})

In [ ]:
image_text['OCR'].lower()

' on this dah 14 years ago warriors nation steph currv itroduced hiiself to the iorld hnd scored 40 poiits aghiist hohzhca clutchpoints valdon:'

In [ ]:
image_text2 = image2text("https://pbs.twimg.com/semantic_core_img/1506780556595326976/qNo3T_to?format=jpg&name=900x900")

In [ ]:
image_text2

{'OCR': ' 80777,273-10 FEon Injecbon Under dose Voe {22 LED Aunnar 7nci Kultiple \' 0l050 "doses',
 'description': 'a knife and a plastic container on a kitchen counter'}

In [ ]:
tweet_text = '''
Australia strikes deal with Moderna to set up mRNA vaccine manufacturing facility in Victoria
Pharmaceuticals giant Moderna is set to build the southern hemisphere's first mRNA vaccine manufacturing facility in Melbourne following a $2-billion, 10-year deal with the Australian and Victorian governments. The facility is expected to manufacture 100 million doses annually, authorities announced on Thursday.
'''
gen_image_hashtags("", image_text2)

' #cooking #kitchen'

In [ ]:
image_text2['description']

'a knife and a plastic container on a kitchen counter'

In [ ]:
image_text3 = image2text("https://pbs.twimg.com/media/FOn7LY5VcAc56PO?format=jpg&name=large")
gen_image_hashtags("", image_text3)

' #miamiheat #nba'

# Midpoint Results

In [ ]:
#data
import pandas as pd
import io
import requests

# data_url = "https://raw.githubusercontent.com/satwikgoyal/Hashtag-Prediction/main/Tweets_Hashtag_Dataset.csv"
# data_url = "https://raw.githubusercontent.com/satwikgoyal/Hashtag-Prediction/main/Tweets%20%2B%20Hashtags%20Dataset%20-%20120.csv"
# data_url = "https://raw.githubusercontent.com/satwikgoyal/Hashtag-Prediction/main/tweet_prompts.csv"
data_url = "https://raw.githubusercontent.com/satwikgoyal/Hashtag-Prediction/main/test_prompts.csv"
s = requests.get(data_url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
df.head(-1)

,Unnamed: 0,prompt,completion
0,1099,"🚀 🚀 🚀 $1,242 , 399 btcusdt longed @ $40,985 . ...",#binance #btc #cryptocurrency
1,327,bitcoin : $41043.9 💔 - 135.46 last 1 hour ( - ...,#bitcoinpriceupdates #bitcoin #hourlycrypto #c...
2,1809,pounding the pavement today . . . https://...,#running #nofilter #poundingpavement #burnener...
3,1622,interactive accessibility meetup happy hour a...,#sxsw #a11y
4,1596,"i bambini hanno bisogno di esser amati , siamo...",#love #tizianoferro
...,...,...,...
1173,495,… ps yo me siento a ver las redes y com...,#laguerra #eldollar #ritmosalvaje #noticias #i...
1174,400,ド文系の大学生がエンジニアになるまでの1年間の記録 / 勉強法 https://t.co/o...,#php #laravel #java #エンジニア #プログラマ
1175,368,the historic ecological and cultural devastati...,#gold #bitcoin #cryptocurrencies #defi #storeo...
1176,1111,thinking about finally investing in cryptocurr...,#bitcoin #etherium #binance #crypto #t0


In [ ]:
#zero shot learning approach
def hashtags(tweet):
  text = '''Find hashtags for the tweet
  
  Tweet: '''
  s = text + tweet + '\nHashtags:'
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=s,
    temperature=0,
    max_tokens=60,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0
  )
  return response["choices"][0]["text"]

In [ ]:
hashtags_result = []
for i in range(len(df['prompt'])):
  hashtags_result.append(hashtags(df['prompt'][i]))

In [ ]:
df['Zero-shot Hashtaghs'] = hashtags_result

In [ ]:
df.head(-1)

,Unnamed: 0,prompt,completion,Zero-shot Hashtaghs
0,1099,"🚀 🚀 🚀 $1,242 , 399 btcusdt longed @ $40,985 . ...",#binance #btc #cryptocurrency,#btc #bitcoin #crypto #cryptocurrency
1,327,bitcoin : $41043.9 💔 - 135.46 last 1 hour ( - ...,#bitcoinpriceupdates #bitcoin #hourlycrypto #c...,#bitcoin #cryptocompare
2,1809,pounding the pavement today . . . https://...,#running #nofilter #poundingpavement #burnener...,"#pounding, #pavement, #today"
3,1622,interactive accessibility meetup happy hour a...,#sxsw #a11y,#interactive #accessibility #meetup #happyhou...
4,1596,"i bambini hanno bisogno di esser amati , siamo...",#love #tizianoferro,#love #children #2022 #gay
...,...,...,...,...
1173,495,… ps yo me siento a ver las redes y com...,#laguerra #eldollar #ritmosalvaje #noticias #i...,\n\n#socialmedia #networking #popcorn
1174,400,ド文系の大学生がエンジニアになるまでの1年間の記録 / 勉強法 https://t.co/o...,#php #laravel #java #エンジニア #プログラマ,#ド文系 #大学生 #エンジニア #勉強法
1175,368,the historic ecological and cultural devastati...,#gold #bitcoin #cryptocurrencies #defi #storeo...,#ecology #culture #sustainability
1176,1111,thinking about finally investing in cryptocurr...,#bitcoin #etherium #binance #crypto #t0,#cryptocurrencies #investing #transactionfees...


In [ ]:
def hit_ratio(actual, predicted):
  accuracy = 0
  count = 0
  for i in range(len(actual)):
    if type(actual[i]) == type(1.0):
      print(i)
    a_set = set(actual[i].lower().split())
    p_set = set(predicted[i].lower().split())
    # print(p_set)
    if len(a_set.intersection(p_set))>0:
      accuracy+=1
  return accuracy/len(actual)

In [ ]:
test = df.dropna().reset_index()

In [ ]:
hit_ratio(test['completion'], test['Zero-shot Hashtaghs'])

0.2666098807495741

In [ ]:
#few-shot approach
def hashtags_fs():
  prompt = 'Find hashtags for the tweet\n\n'
  test_tweets = []
  test_hashtags = []
  for i in range(len(df['Text'])):
    if i%2==1:
      prompt += 'Tweet: ' + df['Text'][i] + '\nHashtags: ' + df['Hashtags'][i] + '\n\n'
    else:
      test_tweets.append(df['Text'][i])
      test_hashtags.append(df['Hashtags'][i])
  pred_tags = []
  for tweet in test_tweets:
    response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt+'Tweet: '+tweet+'\nHashtags:',
    temperature=0,
    max_tokens=60,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0)
    pred_tags.append(response["choices"][0]["text"])
  
  return pred_tags, test_hashtags

In [ ]:
pred_tags, test_hashtags = hashtags_fs()

In [ ]:
hit_ratio(test_hashtags, pred_tags)

0.7333333333333333

In [ ]:
import numpy as np

lens = 0
min_no = 1000
max_no = 0
for i in range(len(df['Zero-shot Hashtaghs'])):
  min_no = np.min([min_no, len(df['Zero-shot Hashtaghs'][i].split())])
  max_no = np.max([max_no, len(df['Zero-shot Hashtaghs'][i].split())])
  lens += len(df['Zero-shot Hashtaghs'][i].split())

lens/=len(df['Zero-shot Hashtaghs'])
lens, min_no, max_no

(3.658333333333333, 2, 20)

In [ ]:
lens = 0
for i in range(len(df['Hashtags'])):
  lens += len(df['Hashtags'][i].split())

lens/=len(df['Hashtags'])
lens

3.475

In [ ]:
df.head(-1)

,Text,Date,Hashtags,Zero-shot Hashtaghs
0,Who wants a free NFT?,03/31/2022,#NFTGivaway #NFT #Competition,#NFT #free
1,🌸Spring giveaway!🌸We're giving away one Spring...,03/31/2022,#Giveaway #Spring #FolkloreThursday #competiti...,#giveaway #spring #books
2,Series 2 of Bridgerton is back today! 😍 To cel...,03/25/2022,#Competition #FF,#Bridgerton #UK
3,*WIN* For your chance to win new active shoes ...,03/23/2022,#win #competition,#giveaway #contest #win #prizes
4,"Throwing it back to Prey, 2007",03/31/2022,#tbt,#Prey #2007
...,...,...,...,...
114,We may be the last generation that has an incr...,03/20/2022,#pressforprogress,#Internet #Generation #Work
115,Celebrate the women who forged the way for you...,03/31/2022,#pressforprogress #WHM2022 #WHM #IWD2022#SeeHe...,#womenshistorymonth #womenshistory #feminism ...
116,Recreate your own version of popular movie and...,03/31/2022,#hollywood #oscars #actor #influencer #influen...,#InVidMe #Recreate #MovieScenes #TVScenes #Cr...
117,"""There's only one Michael Jackson, I'll be the...",03/31/2022,#influencer,#BrunoMars #MichaelJackson


In [ ]:
pred_tags[0]

' #NFT #NFTdrop #NFTs #NFTgiveaway'

In [ ]:
pred_tags[4]

' #travel #vacation #adventure #explore #wanderlust'

In [ ]:
df['Text'][57]

'This is the Belarusian battalion named after Kastus Kalinouski in Ukraine. Belarusian heroes who are helping Ukrainians defend themselves. I have friends there and I know that these people are ready to risk their lives to protect both Ukraine and Belarus'

# Accuracy, Precision, Recall, F-Score

In [ ]:
def get_me_my_metrics(test, predicted):
  accuracy = 0
  precision = 0
  recall = 0
  for i in range(len(test)):
    a_set = set(test[i].lower().split())
    p_set = set(predicted[i].lower().replace('\n', ' ').split())
    # print(p_set)
    common_tags = a_set.intersection(p_set)
    if len(common_tags)>0:
      accuracy+=1
    precision += len(common_tags)/len(a_set)
    recall += len(common_tags)/len(p_set)
  
  accuracy = accuracy/len(test)
  precision = precision/len(test)
  recall = recall/len(test)
  f_score = 2*((precision*recall)/(precision+recall))

  return accuracy, precision, recall, f_score

In [ ]:
accuracy, precision, recall, f_score = get_me_my_metrics(df['Hashtags'], df['Zero-shot Hashtaghs'])
accuracy, precision, recall, f_score

(0.5916666666666667,
 0.3722000684500685,
 0.23843253968253955,
 0.29066448928090216)

In [ ]:
accuracy, precision, recall, f_score = get_me_my_metrics(test_hashtags, pred_tags)
accuracy, precision, recall, f_score

(0.7333333333333333,
 0.4930952380952379,
 0.2706032877797584,
 0.34943944003095995)

# Final Results

In [ ]:
import pandas as pd
import io
import requests

data_url = "https://raw.githubusercontent.com/satwikgoyal/Hashtag-Prediction/main/tweet_prompts.csv"
s = requests.get(data_url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
df.head()

,Unnamed: 0,prompt,completion
0,0,starfish beat jmarkz87 in a head to head chall...,#itsmychance #prizes #1stplace #competition
1,1,very good project wow @ @platonkoreanis pretty...,#competition
2,2,"wow @ @platonkoreanis pretty impressive , test...",#competition
3,3,"very good project , i wish you more success in...",#competition
4,4,this wonderful project . @sumas72181326 @ripon...,#competition


In [ ]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
df.head()

,prompt,completion
0,starfish beat jmarkz87 in a head to head chall...,#itsmychance #prizes #1stplace #competition
1,very good project wow @ @platonkoreanis pretty...,#competition
2,"wow @ @platonkoreanis pretty impressive , test...",#competition
3,"very good project , i wish you more success in...",#competition
4,this wonderful project . @sumas72181326 @ripon...,#competition


In [ ]:
count = 0
for i in range(len(df['completion'])):
  if type(df['completion'][i]) == type(''):
    count += len(df['completion'][i].split())

print('Average Hashtags:', count/len(df['prompt']))

Average Hashtags: 4.0870672097759675


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.6)

In [ ]:
test.head()

,prompt,completion
1099,"🚀 🚀 🚀 $1,242 , 399 btcusdt longed @ $40,985 . ...",#binance #btc #cryptocurrency
327,bitcoin : $41043.9 💔 - 135.46 last 1 hour ( - ...,#bitcoinpriceupdates #bitcoin #hourlycrypto #c...
1809,pounding the pavement today . . . https://...,#running #nofilter #poundingpavement #burnener...
1622,interactive accessibility meetup happy hour a...,#sxsw #a11y
1596,"i bambini hanno bisogno di esser amati , siamo...",#love #tizianoferro


In [ ]:
train.head()

,prompt,completion
399,mais uma vaga quentinha ! ! ! pr : 8812 link :...,#backendbrvagas #clt #python #java #remoto #git
1543,https://t.co/koogvwmbjm,#art #art
1826,enter to win a $50 + makeup prize pack + journ...,#giveaway #makeup #books
1664,goodnight to the bad guy https://t.co/qlbf2...,#scotthall #wcw #ripscotthall
563,it ’ s sad that women are still not being prot...,#endgenderbasedviolence #metoo #istandwithwomen


In [ ]:
train.to_csv("train_prompts.csv")

In [ ]:
from google.colab import files

files.download('train_prompts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test.to_csv("test.csv")
files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Final Image Experiments

In [ ]:
import pandas as pd
import io
import requests

url = data_url = "https://raw.githubusercontent.com/satwikgoyal/Hashtag-Prediction/main/Image_Dataset.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
df.head()

,Text,Image Link,Date,Hashtags,Tweet URL
0,It's so interesting how powerful our utterance...,https://pbs.twimg.com/media/FReNluRXMAEP4C0?fo...,04/29/2022,#Blessed #PlusOne #ThoughtsBecomeThings,https://twitter.com/jessicaosgh/status/1519839...
1,Blessed to be ranked⭐️⭐️⭐️⭐️,https://pbs.twimg.com/media/FRh60UFXEAA32Tm?fo...,04/29/2022,#AGTG #Blessed #Ripgranny,https://twitter.com/OttisjonesIV/status/152009...
2,And so so emotional right now.\nJust THANK YOU.🙏🏻,https://pbs.twimg.com/media/FRg2YPzagAEqEyT?fo...,04/29/2022,#Grateful #Blessed,https://twitter.com/RukminiMaitra/status/15200...
3,Plan the direction you want to go.,https://pbs.twimg.com/media/FRxd8BhWYAA3rxy?fo...,05/02/2022,#determination #motivation #lifestyle #strong ...,https://twitter.com/OshunPublicaton/status/152...
4,Developing your organisation’s strategic think...,https://pbs.twimg.com/media/FRxd9jEXsAYvoeI?fo...,05/02/2022,#Strategy #Leadership #Goals #Planning #Skills,https://twitter.com/interchangepd/status/15211...


In [ ]:
#zero shot learning approach
def hashtags(text, image_url):
  prompt = '''Find hashtags for the Twitter Content
  
  Tweet Content: '''
  image_text = image2text(image_url)
  prompt +=  text + ' ' + image_text['description'] + ' ' + image_text['OCR'] + '\nHashtags:'
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    temperature=0,
    max_tokens=60,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0
  )
  return response["choices"][0]["text"]

In [ ]:
hashtag_list = []

for i in range(len(df['Text'])):
  print(i)
  tags = hashtags(df['Text'][i], df['Image Link'][i])
  hashtag_list.append(tags)

In [ ]:
df['davinci'] = hashtag_list

In [ ]:
from google.colab import files

df.to_csv("generated_hashtags.csv")
files.download('generated_hashtags.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.head()

,Text,Image Link,Date,Hashtags,Tweet URL,davinci
0,It's so interesting how powerful our utterance...,https://pbs.twimg.com/media/FReNluRXMAEP4C0?fo...,04/29/2022,#Blessed #PlusOne #ThoughtsBecomeThings,https://twitter.com/jessicaosgh/status/1519839...,#power #utterances #gratitude
1,Blessed to be ranked⭐️⭐️⭐️⭐️,https://pbs.twimg.com/media/FRh60UFXEAA32Tm?fo...,04/29/2022,#AGTG #Blessed #Ripgranny,https://twitter.com/OttisjonesIV/status/152009...,\n\n#blessed #collage #photos #blue #shirt #lt...
2,And so so emotional right now.\nJust THANK YOU.🙏🏻,https://pbs.twimg.com/media/FRg2YPzagAEqEyT?fo...,04/29/2022,#Grateful #Blessed,https://twitter.com/RukminiMaitra/status/15200...,#emotional #thankful #congratulations #team #...
3,Plan the direction you want to go.,https://pbs.twimg.com/media/FRxd8BhWYAA3rxy?fo...,05/02/2022,#determination #motivation #lifestyle #strong ...,https://twitter.com/OshunPublicaton/status/152...,#goals #plan #success #vehicle #route
4,Developing your organisation’s strategic think...,https://pbs.twimg.com/media/FRxd9jEXsAYvoeI?fo...,05/02/2022,#Strategy #Leadership #Goals #Planning #Skills,https://twitter.com/interchangepd/status/15211...,#businessplanning #strategictthinking #KCHANG...


In [ ]:
get_me_my_metrics(df['Hashtags'], df['davinci'])

(0.48322147651006714,
 0.2233231161083509,
 0.12846825061590159,
 0.16310764141645337)

In [ ]:
  df['Hashtags'][4]

'#Strategy #Leadership #Goals #Planning #Skills'

In [ ]:
len(hashtag_list)

149